# Training a neural network for image classification on MNIST with Keras


### Disclaimer: 
The goal of using this notebook is not intended to learn how to create an ML model, we hope you already know that, instead we want to use this to learn how to deploy an ML model on a gRPC Server as a microservice.

This notebook is downloaded from [Here](https://www.tensorflow.org/datasets/keras_example#step_2_create_and_train_the_model) and I don't have any rights over it.

Copyright 2020 The TensorFlow Datasets Authors, Licensed under the Apache License, Version 2.0

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

## Step 1: Create your input pipeline

Start by building an efficient input pipeline using advices from:
* The [Performance tips](https://www.tensorflow.org/datasets/performances) guide
* The [Better performance with the `tf.data` API](https://www.tensorflow.org/guide/data_performance#optimize_performance) guide


### Load a dataset

Load the MNIST dataset with the following arguments:

* `shuffle_files=True`: The MNIST data is only stored in a single file, but for larger datasets with multiple files on disk, it's good practice to shuffle them when training.
* `as_supervised=True`: Returns a tuple `(img, label)` instead of a dictionary `{'image': img, 'label': label}`.

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

### Build a training pipeline

Apply the following transformations:

* `tf.data.Dataset.map`: TFDS provide images of type `tf.uint8`, while the model expects `tf.float32`. Therefore, you need to normalize images.
* `tf.data.Dataset.cache` As you fit the dataset in memory, cache it before shuffling for a better performance.<br/>
__Note:__ Random transformations should be applied after caching.
* `tf.data.Dataset.shuffle`: For true randomness, set the shuffle buffer to the full dataset size.<br/>
__Note:__ For large datasets that can't fit in memory, use `buffer_size=1000` if your system allows it.
* `tf.data.Dataset.batch`: Batch elements of the dataset after shuffling to get unique batches at each epoch.
* `tf.data.Dataset.prefetch`: It is good practice to end the pipeline by prefetching [for performance](https://www.tensorflow.org/guide/data_performance#prefetching).

In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

### Build an evaluation pipeline

Your testing pipeline is similar to the training pipeline with small differences:

 * You don't need to call `tf.data.Dataset.shuffle`.
 * Caching is done after batching because batches can be the same between epochs.

In [ ]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

## Step 2: Create and train the model

Plug the TFDS input pipeline into a simple Keras model, compile the model, and train it.

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

In [ ]:
model.save('mnist_model')

In [1]:
import tensorflow as tf

In [2]:
def normalize_img(image):
    return tf.cast(image, tf.float32) / 255.0

In [7]:
model = tf.keras.models.load_model('mnist_model')
image = tf.keras.preprocessing.image.load_img('./test_image12.png')
image_tensor = tf.keras.preprocessing.image.img_to_array(image)
print(image_tensor.shape)
image_tensor_gray = tf.image.rgb_to_grayscale(image_tensor)
image_tensor_gray = tf.reshape(image_tensor_gray, [1, 28, 28])
image_tensor_gray = normalize_img(image_tensor_gray)


(28, 28, 3)


In [8]:
result = model.predict(image_tensor_gray)

1/1 [==============================] - 2s 2s/step


In [9]:
print(result.shape)

(1, 10)


In [11]:
print(f"Guess: {result[0].argmax()}\tConfidence: {result[0][result[0].argmax()]*100:.3f}%")


Guess: 5	Confidence: 82.867%


Portions of this page are reproduced from work created and shared by Google and used according to terms described in the Creative Commons 4.0 Attribution License.